## Genereate and Import Data

Elimizde gerçek bir data olmadığı için işimize yarayacak satın alma geçmişi, ürün detayları ve müşteri etkileşimi gibi dataları kendimiz oluşturarak import ediyoruz. 
Data oluşturma işlemleri kalabalık görünmemesi için bunun için bir fonkisyon yazdım,
data_generate.py içerisinden build_data() fonkiyonunu çağırarak hızlıca yapabiliriz.

In [20]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import data_generate # Data oluşturma işlemleri için hazırladığım kütüphane

In [2]:
# Aynı dataların oluşabilmesi için random_state=42 ayarlandı.
customer_interactions_df, purchase_history_df, product_details_df = data_generate.build_data(random_state=42)

In [3]:
customer_interactions_df.head(), purchase_history_df.head(), product_details_df.head()

(   customer_id  book_id  page_views  view_dates
 0           52       21           1  2023-07-28
 1           93        4           1  2023-09-12
 2           15       18           1  2023-06-18
 3           72       14           1  2023-08-12
 4           61       28           1  2023-06-16,
    customer_id  book_id purchase_dates
 0           88        1     2023-12-07
 1           11       21     2023-08-25
 2           62       14     2023-10-04
 3           72        1     2023-12-01
 4           15       10     2023-07-19,
    book_id                                        book_name   price  rating
 0        1             Reactive foreground capacity of Onto  157.47     3.0
 1        2          Universal national framework of Student  131.06     1.9
 2        3  Profit-focused content-based structure of Sound  463.90     2.0
 3        4      Optional composite collaboration of Surface  446.89     2.6
 4        5   Switchable bi-directional methodology of Range  344.81     2.5)

Literatürde tavsiye sistemleri için kullanılan bir çok yöntem var. 
Bunlardan bazıları kullanıcı tabanlı, bazıları ürün ya da içerik tabanlı, bazıları ise pattern ya da kural tabanlı olabilir.

Datamıza uygun olan ve en sık kullanılan yöntemlerden birkaçını deneyeceğim. Yöntemler arasındaki farklar ise kısaca aşağıda belirtilmiştir.

![Example Image](comparison_table.png)



## Collaborative Filtering

Collaborative Filtering'i bu örnekte 3 farklı şekilde yapabiliriz.

1. Item Based: Ürün benzerliğinden yola çıkarak tavsiyede bulunuruz. 
2. User Based : Kullanıcı benzerliğinden yola çıkarak tavsiyede bulunuruz.
3. Content Based : Ürün içeriği kapsayıcılığından yola çıkarak tavsiyede bulunuruz.

## User Based Recommendation

In [5]:
df = pd.merge(purchase_history_df, product_details_df, on='book_id')
df

,customer_id,book_id,purchase_dates,book_name,price,rating
0,88,1,2023-12-07,Reactive foreground capacity of Onto,157.47,3.0
1,72,1,2023-12-01,Reactive foreground capacity of Onto,157.47,3.0
2,21,1,2023-08-21,Reactive foreground capacity of Onto,157.47,3.0
3,27,1,2023-12-29,Reactive foreground capacity of Onto,157.47,3.0
4,79,1,2023-11-05,Reactive foreground capacity of Onto,157.47,3.0
...,...,...,...,...,...,...
9995,7,13,2023-07-17,Fundamental secondary access of Would,226.68,1.6
9996,57,13,2023-07-03,Fundamental secondary access of Would,226.68,1.6
9997,32,13,2023-07-09,Fundamental secondary access of Would,226.68,1.6
9998,90,13,2023-06-27,Fundamental secondary access of Would,226.68,1.6


First, we will create a user_item_matrix to observe the relationship between each customer and the products they have purchased.

The creation of a user-item matrix involves representing the interaction between users and items in a matrix format, where rows represent users, columns represent items, and the entries of the matrix denote the interaction (e.g., total purchase, rating) between users and items.

In [26]:
# Adım 1: Kullanıcı ve ürün matrisini oluşturalım.

user_item_matrix = df.pivot_table(index='customer_id', columns='book_id', values='rating', aggfunc='count', fill_value=0)

user_item_matrix

book_id,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
customer_id,,,,,,,,,,,,,,,,,,,,,
1,2,3,2,3,3,2,3,5,7,4,...,5,4,2,2,8,4,3,2,4,3
2,6,3,4,7,5,1,1,3,6,1,...,6,4,4,2,4,4,7,3,6,6
3,0,0,3,1,3,4,3,5,5,1,...,3,5,0,5,4,5,1,4,2,2
4,5,4,5,4,2,4,3,2,7,0,...,4,4,2,3,3,2,3,4,2,4
5,2,6,3,4,4,3,3,0,2,2,...,2,4,5,4,6,1,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4,2,3,3,4,2,7,1,5,5,...,4,3,3,5,2,2,4,5,3,5
97,3,5,2,1,2,2,1,6,5,5,...,6,4,3,2,1,4,3,4,0,2
98,1,8,4,7,4,3,3,9,2,7,...,5,3,2,4,3,5,2,3,8,3


Kullanıcılar arasındaki kosinüs benzerliğini hesaplayarak, benzer tercihlere ve davranışlara sahip kullanıcıları belirleyebiliriz. Bu sayede, benzer kullanıcıların tercihlerine dayanarak öneriler yapabiliriz. Bu yöntem, kullanıcı tabanlı iş birliğine dayalı filtreleme (user-based collaborative filtering) yönteminin temelini oluşturur.

In [33]:
# Adım 2: Cosinüs Benzerliğini hesaplayalım

user_similarity_matrix = cosine_similarity(user_item_matrix)
user_similarity_matrix

array([[1.        , 0.84960174, 0.75856321, ..., 0.81389588, 0.71240354,
        0.79356346],
       [0.84960174, 1.        , 0.71792078, ..., 0.79761615, 0.84060375,
        0.83139886],
       [0.75856321, 0.71792078, 1.        , ..., 0.72700104, 0.73927145,
        0.7761088 ],
       ...,
       [0.81389588, 0.79761615, 0.72700104, ..., 1.        , 0.75952008,
        0.80599727],
       [0.71240354, 0.84060375, 0.73927145, ..., 0.75952008, 1.        ,
        0.80377904],
       [0.79356346, 0.83139886, 0.7761088 , ..., 0.80599727, 0.80377904,
        1.        ]])

In [46]:
# Adım 3: Bu benzerliklere göre kullanıcıya ürün önerisi yapan bir fonkisyon yazalım.

def recommend_products_user(user_id, num_recommendations=6):
    # İlgili kullanıcı matrix içinde bulunup filtrelenir
    user_index = user_item_matrix.index.get_loc(user_id)
    
    # Cosinüs benzerliğine göre benzer kullanıcılar bulunur
    similar_users = user_similarity_matrix[user_index]
    similar_users_index = similar_users.argsort()[-7:-1][::-1]  # En benzer 6 kullanıcı
    
    # Benzer kullanıcılar tarafından satın alınan ürünlerin listesi getirilir.
    similar_products = []
    for user_index in similar_users_index:
        similar_product_index = list(user_item_matrix.loc[user_index,:][user_item_matrix.loc[user_index,:] > 0].index)
        similar_products += similar_product_index
    
    # Benzer ürünlerin sipariş sayısı saydırılır
    similar_products = list(set(similar_products))
    similar_products_agg = [(product_id, user_item_matrix[product_id].sum()) for product_id in similar_products]
    
    # Bu ürünler adetlerine göre sıralanır
    sorted_similar_products = sorted(similar_products_agg, key=lambda x: x[1], reverse=True)
    similar_products_sorted = [product_id for (product_id, count) in sorted_similar_products]
    
    # Kullanıcıya satın alması için en iyi ürünler tavsiye edilir. (daha önce almamış olduğu ürünler)
    user_product_index = list(user_item_matrix.loc[user_id,:][user_item_matrix.loc[user_id,:] > 0].index)
    recommended_products = [product_id for product_id in similar_products_sorted if product_id not in user_product_index]

    return recommended_products[:num_recommendations]

In [47]:
# Example: 3 numaralı kullanıcı id'si için ürün önerisi yapalım
user_id = 3
recommended_products = recommend_products_user(user_id)

print(f"Recommended Products for {user_id}:")
for product_id in recommended_products:
    product_info = product_details_df[product_details_df['book_id'] == product_id]
    print(product_info[['book_id', 'book_name', 'price']].values[0])

Recommended Products for 3:
[2 'Universal national framework of Student' 131.06]
[1 'Reactive foreground capacity of Onto' 157.47]
[23 'Vision-oriented 24/7 toolset of Hold' 472.08]


In [48]:
recommend_products_user(3)

[2, 1, 23]

## Item Based Recommendation

User_item_matrix'in transpozesini alarak, kosinüs benzerliği gibi yöntemlerle öğeler arasındaki benzerlikleri kolayca hesaplayabiliriz. Bu sayede, kullanıcıların geçmişte etkileşimde bulunduğu benzer öğelere dayalı olarak öneriler sunabiliriz.

In [41]:
item_user_matrix = user_item_matrix.transpose()
item_user_matrix.head()

customer_id,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
book_id,,,,,,,,,,,,,,,,,,,,,
1,2,6,0,5,2,5,2,2,4,2,...,0,2,5,0,6,4,3,1,4,2
2,3,3,0,4,6,3,4,3,2,4,...,6,3,2,2,4,2,5,8,5,3
3,2,4,3,5,3,3,4,6,5,6,...,1,2,4,1,4,3,2,4,5,3
4,3,7,1,4,4,2,4,3,4,4,...,1,4,0,2,4,3,1,7,4,3
5,3,5,3,2,4,3,4,5,5,3,...,1,4,1,4,2,4,2,4,3,3


In [43]:
# Adım 1: Ürünler arasındaki cosinüs benzerliğini hesaplayalım
item_similarity_matrix = cosine_similarity(item_user_matrix)
item_similarity_matrix

array([[1.        , 0.74030323, 0.79211434, 0.717697  , 0.76597241,
        0.80353413, 0.76611563, 0.73829928, 0.73997179, 0.7318816 ,
        0.74762614, 0.68583324, 0.74778357, 0.7459049 , 0.76117226,
        0.75852362, 0.75122248, 0.76671158, 0.73263148, 0.7614032 ,
        0.73685452, 0.78492795, 0.77499813, 0.75350615, 0.69504884,
        0.70308383, 0.75618584, 0.77765317, 0.75542048, 0.77174741],
       [0.74030323, 1.        , 0.76287854, 0.82179457, 0.81357447,
        0.80909632, 0.76457729, 0.77705742, 0.78583604, 0.80144333,
        0.81815331, 0.72883478, 0.81429041, 0.77812566, 0.78595792,
        0.81366234, 0.83018375, 0.78469064, 0.78068439, 0.81683804,
        0.75925016, 0.82231137, 0.76405518, 0.79189971, 0.75760901,
        0.76098702, 0.78822787, 0.77885441, 0.75143177, 0.81777004],
       [0.79211434, 0.76287854, 1.        , 0.76949837, 0.78448919,
        0.77146004, 0.78541221, 0.7538225 , 0.78495165, 0.74303635,
        0.79396905, 0.71375795, 0.77200915, 0.

In [44]:
# Adım 2: Bu benzerliklere göre bir ürün satın alındığında ya da görüntülendiğinde kullanıcıya ürün önerelim

def recommend_products_item(item_id, num_recommendations=6):
    # Find the index of the item in the item-user matrix
    item_index = item_user_matrix.index.get_loc(item_id)
    
    # Find similar items based on cosine similarity
    similar_items = item_similarity_matrix[item_index]
    similar_items_index = similar_items.argsort()[-7:-1][::-1]  # Extract top 6 similar items
    
    # Recommend similar products
    recommended_products = [item_id for item_id in similar_items_index]
    
    return recommended_products[:num_recommendations]

In [45]:
# Example: Recommend products to a Product with ID 7
item_id=7
recommended_products = recommend_products_item(item_id)

print(f"Recommended Products for Product {item_id}: {product_details_df[product_details_df['book_id'] == 1].values[0]}")
for product_id in recommended_products:
    product_info = product_details_df[product_details_df['book_id'] == product_id]
    print(product_info[['book_id', 'book_name', 'price']].values[0])

Recommended Products for Product 7: [1 'Reactive foreground capacity of Onto' 157.47 3.0]
[14 'Re-engineered executive product of Certainly' 403.19]
[10 'Diverse clear-thinking approach of Save' 449.15]
[5 'Switchable bi-directional methodology of Range' 344.81]
[15 'Multi-channeled solution-oriented Internet solution of Continue'
 423.97]
[9 'Cloned clear-thinking methodology of Remember' 387.18]
[2 'Universal national framework of Student' 131.06]


## Ek olark Content Based ürün tavsiyesinde de bulunabiliriz. Fakat bunun için gerçek ürün içeriklerine, özelliklerine ya açıklamalarına ihtiyacımız var. Bu sebeple 3 farklı tipte örnek yaparak bu projeyi kapatıyorum.